# EAS 503 Final Project
**Name:** Sushrut Ishwar Gaikwad

**UB Person Number:** 50604159

## Create a Normalized Database (3NF)

In [1]:
# Imports
import csv
import sqlite3

In [3]:
raw_data_path = "data\\raw\\adult.csv"
normalized_db_path = "data\\normalized\\normalized.db"

def create_and_populate_database(raw_data_path, normalized_db_path):
    # Connect to SQLite database
    conn = sqlite3.connect(normalized_db_path)
    cursor = conn.cursor()
    
    # Create normalized tables
    cursor.execute("CREATE TABLE IF NOT EXISTS workclass (id INTEGER PRIMARY KEY, name TEXT UNIQUE)")
    cursor.execute("CREATE TABLE IF NOT EXISTS education (id INTEGER PRIMARY KEY, name TEXT UNIQUE)")
    cursor.execute("CREATE TABLE IF NOT EXISTS marital_status (id INTEGER PRIMARY KEY, name TEXT UNIQUE)")
    cursor.execute("CREATE TABLE IF NOT EXISTS occupation (id INTEGER PRIMARY KEY, name TEXT UNIQUE)")
    cursor.execute("CREATE TABLE IF NOT EXISTS relationship (id INTEGER PRIMARY KEY, name TEXT UNIQUE)")
    cursor.execute("CREATE TABLE IF NOT EXISTS race (id INTEGER PRIMARY KEY, name TEXT UNIQUE)")
    cursor.execute("CREATE TABLE IF NOT EXISTS gender (id INTEGER PRIMARY KEY, name TEXT UNIQUE)")
    cursor.execute("CREATE TABLE IF NOT EXISTS native_country (id INTEGER PRIMARY KEY, name TEXT UNIQUE)")
    cursor.execute("CREATE TABLE IF NOT EXISTS income (id INTEGER PRIMARY KEY, name TEXT UNIQUE)")
    
    # Create `main_data` table with foreign keys
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS main_data (
            id INTEGER PRIMARY KEY,
            age INTEGER,
            workclass_id INTEGER,
            fnlwgt INTEGER,
            education_id INTEGER,
            educational_num INTEGER,
            marital_status_id INTEGER,
            occupation_id INTEGER,
            relationship_id INTEGER,
            race_id INTEGER,
            gender_id INTEGER,
            capital_gain INTEGER,
            capital_loss INTEGER,
            hours_per_week INTEGER,
            native_country_id INTEGER,
            income_id INTEGER,
            FOREIGN KEY (workclass_id) REFERENCES workclass (id),
            FOREIGN KEY (education_id) REFERENCES education (id),
            FOREIGN KEY (marital_status_id) REFERENCES marital_status (id),
            FOREIGN KEY (occupation_id) REFERENCES occupation (id),
            FOREIGN KEY (relationship_id) REFERENCES relationship (id),
            FOREIGN KEY (race_id) REFERENCES race (id),
            FOREIGN KEY (gender_id) REFERENCES gender (id),
            FOREIGN KEY (native_country_id) REFERENCES native_country (id),
            FOREIGN KEY (income_id) REFERENCES income (id)
        )
    """)
    
    # Read data from CSV
    with open(raw_data_path, 'r') as infile:
        reader = csv.DictReader(infile)
        
        # Helper function to insert into normalized tables and get IDs
        def get_or_insert(table, name):
            cursor.execute(f"INSERT OR IGNORE INTO {table} (name) VALUES (?)", (name,))
            cursor.execute(f"SELECT id FROM {table} WHERE name = ?", (name,))
            return cursor.fetchone()[0]
        
        # Insert data into normalized tables and `main_data`
        for row in reader:
            workclass_id = get_or_insert('workclass', row['workclass'])
            education_id = get_or_insert('education', row['education'])
            marital_status_id = get_or_insert('marital_status', row['marital-status'])
            occupation_id = get_or_insert('occupation', row['occupation'])
            relationship_id = get_or_insert('relationship', row['relationship'])
            race_id = get_or_insert('race', row['race'])
            gender_id = get_or_insert('gender', row['gender'])
            native_country_id = get_or_insert('native_country', row['native-country'])
            income_id = get_or_insert('income', row['income'])
            
            # Insert into main_data table
            cursor.execute("""
                INSERT INTO main_data (
                    age, workclass_id, fnlwgt, education_id, educational_num,
                    marital_status_id, occupation_id, relationship_id, race_id,
                    gender_id, capital_gain, capital_loss, hours_per_week,
                    native_country_id, income_id
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, (
                row['age'], workclass_id, row['fnlwgt'], education_id,
                row['educational-num'], marital_status_id, occupation_id,
                relationship_id, race_id, gender_id, row['capital-gain'],
                row['capital-loss'], row['hours-per-week'], native_country_id,
                income_id
            ))

    # Commit and close the connection
    conn.commit()
    conn.close()

# Populate the SQLite database
create_and_populate_database(raw_data_path, normalized_db_path)

## Write SQL Join Statement to Fetch the Data from the Database and into Pandas DataFrame

In [4]:
# Imports
import pandas as pd

In [5]:
def reconstruct_data(normalized_db_path):
    # Connect to SQLite database
    conn = sqlite3.connect(normalized_db_path)
    query = """
        SELECT 
            main_data.age,
            workclass.name AS workclass,
            main_data.fnlwgt,
            education.name AS education,
            main_data.educational_num AS educational_num,
            marital_status.name AS marital_status,
            occupation.name AS occupation,
            relationship.name AS relationship,
            race.name AS race,
            gender.name AS gender,
            main_data.capital_gain AS capital_gain,
            main_data.capital_loss AS capital_loss,
            main_data.hours_per_week AS hours_per_week,
            native_country.name AS native_country,
            income.name AS income
        FROM main_data
        JOIN workclass ON main_data.workclass_id = workclass.id
        JOIN education ON main_data.education_id = education.id
        JOIN marital_status ON main_data.marital_status_id = marital_status.id
        JOIN occupation ON main_data.occupation_id = occupation.id
        JOIN relationship ON main_data.relationship_id = relationship.id
        JOIN race ON main_data.race_id = race.id
        JOIN gender ON main_data.gender_id = gender.id
        JOIN native_country ON main_data.native_country_id = native_country.id
        JOIN income ON main_data.income_id = income.id
    """
    
    # Execute query and fetch data into a DataFrame
    reconstructed_data = pd.read_sql_query(query, conn)
    conn.close()
    
    return reconstructed_data

df = reconstruct_data(normalized_db_path)

reconstruct_csv_path = "data\\reconstructed\\reconstructed.csv"
df.to_csv(reconstruct_csv_path, index=False)

df.head()

,age,workclass,fnlwgt,education,educational_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
